https://poke-env.readthedocs.io/en/stable/modules/battle.html

classpoke_env.environment.abstract_battle.AbstractBattle(battle_tag: str, username: str, logger: Logger, save_replays: str | bool, gen: int)

On va tenter de parser ce replay : https://replay.pokemonshowdown.com/gen8randombattle-1123651831.log afin d'en obtenir un objet battle (enfin un seul avec ses différents états à chaque tour). Ensuite, lancer le battle (on verra comment) pour vérifier que ça marche bien. Fichier contenu dans "data/first_replay.txt"

In [1]:
import poke_env

In [2]:
!which python

/Users/dan2/.pyenv/versions/venv/bin/python


In [3]:
import pandas as pd
from poke_env.environment.battle import Battle

In [9]:
from poke_env.environment.battle import Battle

with open("data/first_replay.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Extraire les lignes depuis |start jusqu'à |turn|2
battle_lines = []
in_battle = False
for line in lines:
    if "|start" in line:
        in_battle = True
    if in_battle:
        battle_lines.append(line.strip())
    if "|turn|2" in line:
        break

# Créer un objet Battle avec un logger vide
battle = Battle("replay-test", "p1", logger=None, gen="8")

# Parser les lignes pour initialiser l'état
for msg in battle_lines:
    try:
        battle.parse_message(msg)
    except Exception as e:
        print(f"Erreur avec {msg}: {e}")

# Affichage des états des Pokémon
print("🟢 Pokémon actif (p1) :")
if battle.active_pokemon:
    print(f"Nom : {battle.active_pokemon.species}")
    print(f"HP : {battle.active_pokemon.current_hp_fraction * 100:.1f}%")
    print(f"Status : {battle.active_pokemon.status}")
else:
    print("Aucun Pokémon actif côté p1")

print("\n🔴 Pokémon adverse (p2) :")
if battle.opponent_active_pokemon:
    print(f"Nom : {battle.opponent_active_pokemon.species}")
    print(f"HP : {battle.opponent_active_pokemon.current_hp_fraction * 100:.1f}%")
    print(f"Status : {battle.opponent_active_pokemon.status}")
else:
    print("Aucun Pokémon actif côté p2")

# Extraire le label (action faite par p1)
label = None
for msg in battle_lines:
    if msg.startswith("|turn|1"):
        in_turn = True
        continue
    elif msg.startswith("|turn|2"):
        break
    elif msg.startswith("|switch|p1a:"):
        label = "switch"
        break
    elif msg.startswith("|move|p1a:"):
        label = "move"
        break

print(f"\n✅ Label pour le tour 1 (action p1) : {label}")





Erreur avec |start: |start
Erreur avec |switch|p1a: Incineroar|Incineroar, L82, F|290/290: |switch|p1a: Incineroar|Incineroar, L82, F|290/290
Erreur avec |switch|p2a: Vaporeon|Vaporeon, L84, M|356/356: |switch|p2a: Vaporeon|Vaporeon, L84, M|356/356
Erreur avec |-ability|p1a: Incineroar|Intimidate|boost: |-ability|p1a: Incineroar|Intimidate|boost
Erreur avec |-unboost|p2a: Vaporeon|atk|1: |-unboost|p2a: Vaporeon|atk|1
Erreur avec |turn|1: |turn|1
Erreur avec |inactive|Battle timer is ON: inactive players will automatically lose when time's up. (requested by sushi8899): |inactive|Battle timer is ON: inactive players will automatically lose when time's up. (requested by sushi8899)
Erreur avec |: string index out of range
Erreur avec |switch|p1a: Rotom|Rotom-Mow, L84|221/221: |switch|p1a: Rotom|Rotom-Mow, L84|221/221
Erreur avec |move|p2a: Vaporeon|Toxic|p1a: Rotom: |move|p2a: Vaporeon|Toxic|p1a: Rotom
Erreur avec |-status|p1a: Rotom|tox: |-status|p1a: Rotom|tox
Erreur avec |: string index

In [1]:
from poke_env.player.random_player import RandomPlayer

player_1 = RandomPlayer(battle_format="gen8randombattle")
player_2 = RandomPlayer(battle_format="gen8randombattle")


In [2]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

async def run_battle():
    await player_1.send_challenges(player_2, n_challenges=1)

    while not player_1.battles:
        await asyncio.sleep(0.1)

    battle = list(player_1.battles.values())[0]

    while battle.turn == 0:
        await asyncio.sleep(0.1)

    vector = player_1.embed_battle(battle)
    print("✅ embed_battle :")
    print("Type :", type(vector))
    print("Shape :", vector.shape)
    print("Extrait :", vector[:10])

await run_battle()





2025-03-24 20:56:44,043 - RandomPlayer 1 - WARNING - Popup message received: |popup|The user '<poke_env.player.random_player.RandomPlayer object at 0x1067ca010>' was not found.


C'est galère, je vais d'abord voir comment fonctionnent les embeddings de vecteurs d'apprentissage et ensuite on rrevient ici. Il semble qu'on a as vraiment besoin de passer du texte à battle, car on utilise ensuite l'objet battle pour construire un vecteur vraiment très simple (avec les mooves, le poke actif, les multiplieurs de dégâts), donc il faut à mon avis juste passer du .txt au vecteur encoding et ça sera peut-être pas si dur. Enfin sauf pour ce qui est de connaitre la puissance des mooves etc.. car elle est pas donné. On va construire plusieurs df : 
- df des sets possibles, des mooves, stats, résistances/faiblesses, de chaque poké du gen8randombattle (car très pratique pour après je pense)
- df des mooves et leurs BP (pour passer du replay au vecteur embbed)